In [1]:
from sympy import symbols, parsing, expand, simplify
from  numpy import poly1d, around
from Interpolation import *
from Helper_functions import *

In [2]:
def Numerical_Diff(x_values, f_values, show_working=False, up = 2):
    Fwd_diff_table = DifferenceArray(f_values)
    exp = input('Which value you want to find: ')
    value_to_find = check_d_n_r(float(exp))

    a = x_values[0]
    h = round(x_values[1] - x_values[0], 5)
    n = len(x_values)
    x = check_d_n_r(round((value_to_find - a)/h, 5))
    j = 0

    div, add, out = '', '+', []
    while n:
        temp, tt, add2 = '', '', ''
        if j>0:
            for i in range(j):
                mul = '*'
                if i: tt += f' {mul} (x - {i})'
                else: tt += f'x'     
                    
        if j: add2 = '+'
        if  j>=1: tt += ' * '
        if j+1 != n and j: div = '/'
        if not j : temp = ''
        else: temp = f'/ {j}!'

        con = f"({sympy.expand('*'.join(tt.split(' * ')[:-1]))}) " if tt else '1'
        con2 = f'/ {fac(j)}' if fac(j)>1 else ''
        
        if Fwd_diff_table[j][0] != '-' and con:
            res = str(sympy.simplify(con)) + f' * {Fwd_diff_table[j][0]} {con2}'
            if res != '0':out.append(res)
        
        if show_working:
            print(add2, tt, end=' ')
            print(f"{Fwd_diff_table[j][0]} {temp}")
            
        j+=1  
        n-=1
        
    if show_working: print('\n = ' + " + ".join(out))
    return out , h, a, x, value_to_find

In [3]:
def cal_derivative(eq, h, a, x, at_x, orderOfDerivative=1, up=5):
    prime = "'"*orderOfDerivative
    add_star = f' ** {orderOfDerivative})' if orderOfDerivative>1 else ')'
    t = ''
    
    for i in range(len(eq)):
        add = ''
        if i+1 != len(eq): add = ' + '
        temp_der = sympy.diff(eq[i], 'x', orderOfDerivative)

        if type(temp_der) in [sympy.core.numbers.Zero, sympy.core.numbers.Float]:
            temp_der = check_d_n_r(temp_der)
        
        t += str(temp_der) + add
    
    derivative = sympy.simplify(t)
    f_a = derivative.subs({'x':x})
    
    print(f"\n  Here a = {a}, h = {h}, x = {at_x}")
    print(f"\n ∴ (h{add_star} * f{prime}(a+xh) = ({t})")
    print(f"\n ∴ f{prime}({at_x}) = ({check_d_n_r(f_a)}) / ({h}{add_star}, at x = {x}")
    print(f"\n ∴ f{prime}({at_x}) = {check_d_n_r(f_a/h**orderOfDerivative)}")
    print('--'*30)

In [4]:
def piece_wise_poly(x_v, f_v, show_working=False):
    calc = ''
    for i in range(len(x_v)):
        t_x_v = x_v.copy()
        if len(t_x_v)>1:
            add = ''
            if i+1 != len(x_v): add='+'
            t_x_v.pop(i)
            ex1 = f'{getFactors(t_x_v, len(t_x_v))} / '
            ex2 = f'({getFactors(t_x_v, len(t_x_v), x_v[i])})  * ({f_v[i]}) {add} '
            calc += ex1 + ex2
            
            if show_working:
                print('\t' + ex1 + ' ' + ex2, end='')
    if show_working: print('\n        = ', end='')
    return sympy.expand(calc)

def Piecewise_Interpolation(x_list, f_values, gap, show_working=False):
    if gap > len(x_list)-1:
        print(f'[-] In Order to find the Polynomial of order {gap}, len(x_list) should be greater than gap')
        return None
    
    intervals = getIntervals(x_list, gap)
    f_intervals = getIntervals(f_values, gap)
    pwi = {1:'Linear', 2:'Quadratic', 3:'Cubic', 4:'Quartic'}
    print(f'By Piecewise {pwi[gap]} Interpolating Formula we have, ')
    
    for _ in range(len(intervals)):
        print("\tBy Lagrange's Interpolation Formula\n\n   f(x) = " , end='\n' if show_working else '')
        print(piece_wise_poly(intervals[_], f_intervals[_], show_working), end=' ')
        print(' x  ∈  [', ", ".join([str(intervals[_][0]), str(intervals[_][-1])]), ']')
        print('----'*15)

In [5]:
def lgrange_Bivariate_I(x_list, alpha='x', write=False, str_format=True):
    result = []
    result_2 = []
    for i in range(len(x_list)):
        denon_str = ''
        num_str = ''
        mul_poly = poly1d([1])
        dr = 1
        for j in range(len(x_list)):
            if i!=j:
                num_str += f'({alpha} - {x_list[j]}) '
                mul_poly *= poly1d([1, -x_list[j]])
                denon_str += f'({x_list[i]} - {x_list[j]})'
                dr *= x_list[i] - x_list[j]
                
        res = mul_poly/dr
        result_2.append(f'{num_str.replace(") (", ") * (")}/ ({dr})')
        num_str = num_str.replace(") (", ') * (')
        
        if write:
            print(f'    {num_str}/({denon_str})')
            print('\t' + " ".join(coef_2_str(res.coef[::-1], alpha)))
            print('---------------------------------------------')
            
        result.append(res.coef[::-1])

    if str_format:
        return result_2
    
    return result


In [6]:
def Lagrange_Bivariate_IF(x_vals, y_vals, arr, show_working=False):
    m, n = len(x_vals), len(y_vals)
    if show_working:
        print(f"Here m: {m}, n: {n}\n")
    
    x__ = lgrange_Bivariate_I(x_vals)
    y__ = lgrange_Bivariate_I(y_vals, 'y')
    
    if show_working:
        print(f"\nBy Lagrange's Bivariate Interpolating Formula \n\nP(x, y) = ", end='')
    for i in range(len(arr)):
        add4, tab = '', ''
        if i: tab = '\t'
        if i: add4 = ' + '
        if show_working:
            print(f'{tab}{add4}X_{m-1},{x_vals[i]} * ',end='[')
        for j in range(len(arr[i])):
            add = ''
            if j+1 != len(arr[i]):
                add = '+'
            if show_working:
                print(f'  Y_{n-1},{y_vals[j]} * (F_{x_vals[i]}_{y_vals[j]}) {add}', end=' ')
        if show_working:
            print(']')
    
    if show_working:
        print('\n= ', end='')
        
    t = ''
    for i in range(len(arr)):
        res_str = ''
        add4 = ''
        
        if i: add4 = ' + '
        if show_working:
            print(f'{add4}({x__[i]}) * ',end='[ ')
            
        res_str += f'({x__[i]}) * ('
        for j in range(len(arr[i])):
            add = ''
            if j+1 != len(arr[i]):
                add = ' + '
            add2 = ''
            if j: add2 = ' + '
            res_str += f'({y__[j]} * {arr[i][j]}) {add}'
            if show_working:
                print(f'({y__[j]} * {arr[i][j]}) {add}', end='')
            
        res_str += ')'
            
        add3 = ''
        if i+1 != len(arr):add3 = ' + '

        t += str(expand(res_str)) + add3
        if show_working:
            print(']')

    if show_working:
        print('\n ∴ P(x, y) = ' , simplify(t))
        
    return simplify(t)



In [7]:
def d_fx_f(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 1)
    return round(((f[i_][y_at] - f[x_at][y_at])/h), n)

def d_fx_b(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1)
    return round(((f[x_at][y_at] - f[i_][y_at])/h), n)

def d_fx_c(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    return round(((f[i_[0]][y_at] - f[i_[1]][y_at])/(2*h)), n)

In [8]:
def d_fy_f(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 1)
    return round(((f[x_at][j_] - f[x_at][y_at])/k), n)

def d_fy_b(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, -1)
    return round(((f[x_at][y_at] - f[x_at][j_])/k), n)

def d_fy_c(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    return round(((f[x_at][j_[0]] - f[x_at][j_[1]])/(2*k)), n)

In [9]:
def d2_fx_f(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, 2), value_at(x_vals, x_at, 1)
    return round((f[i_[0]][y_at] - 2*f[i_[1]][y_at] + f[x_at][y_at])/h**2, n)

def d2_fx_b(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1), value_at(x_vals, x_at, -2)
    return round((f[x_at][y_at] - 2*f[i_[i_[0]]][y_at] + f[i_[1]][y_at])/h**2, n)

def d2_fx_c(x_vals, x_at, y_at, h, f, n=5):
    i_ = value_at(x_vals, x_at, -1), value_at(x_vals, x_at, 1)
    return round((f[i_[0]][y_at] - 2*f[x_at][y_at] + f[i_[1]][y_at])/h**2, n)

In [10]:
def d2_fy_f(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, 2), value_at(y_vals, y_at, 1)
    return round((f[x_at][j_[0]] - 2*f[x_at][j_[1]] + f[x_at][y_at])/k**2, n)

def d2_fy_b(y_vals, x_at, y_at, k, f, n=5):
    j_ = value_at(y_vals, y_at, -1), value_at(y_vals, y_at, -2)
    return round((f[x_at][y_at] - 2*f[x_at][j_[0]] + f[x_at][j_[1]])/k**2, n)

def d2_fy_c(y_vals, x_at, y_at, f, a, n=5):
    j_ = value_at(y_vals, y_at, -1), value_at(y_vals, y_at, 1)
    return round((f[x_at][j_[0]] - 2*f[x_at][y_at] + f[x_at][j_[1]])/k**2, n)

In [11]:
def d2_fxy(x_vals, y_vals, x_at, y_at, h, k, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    nr = f[i_[0]][j_[0]] - f[i_[0]][j_[1]] - f[i_[1]][j_[0]] + f[i_[1]][j_[1]]
    dnr = 4 * h * k
    return round(nr/dnr, n)

def d2_fyx(x_vals, y_vals, x_at, y_at, h, k, f, n=5):
    i_ = value_at(x_vals, x_at, 1), value_at(x_vals, x_at, -1)
    j_ = value_at(y_vals, y_at, 1), value_at(y_vals, y_at, -1)
    nr = f[i_[0]][j_[0]] - f[i_[0]][j_[1]] - f[i_[1]][j_[0]] + f[i_[1]][j_[1]]
    dnr = 4 * h * k
    return round(nr/dnr, n)

In [12]:
def del_x_j(x_0, y_0, h, f):
    return check_d_n_r((f[x_0 + h][y_0] - f[x_0][y_0])/(2*h))

def del_y_j(x_0, y_0, k, f):
    return check_d_n_r((f[x_0][y_0+k] - f[x_0][y_0])/(2*k))

def del_x(x_0, y_0, h, f):
    print(x_0, y_0, h, f)
    if type(f)==dict:
        return f[x_0 + h][y_0] - f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0+h, y_0) - sub_vals(f, x_0, y_0)
    
def del_y(x_0, y_0, k, f):
    if type(f)==dict:
        return f[x_0][y_0+k] - f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0, y_0+k) - sub_vals(f, x_0, y_0)
    
def del_xx(x_0, y_0, h, f, n=5):
    c1, c2 = round(x_0 + 2* h, n), round(x_0+h, n)
    if type(f)==dict:
        return f[c1][y_0] - 2 * f[c2][y_0] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, c1, y_0) - 2 * sub_vals(f, c2, y_0) + sub_vals(f, x_0, y_0)

def del_yy(x_0, y_0, k, f, n=5):
    c1, c2 = round(y_0+2*k, n), round(y_0+k, n)
    if type(f)==dict:
        return f[x_0][c1] - 2 * f[x_0][c2] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, x_0, c1) - 2 * sub_vals(f, x_0, c2) + sub_vals(f, x_0, y_0)

def del_xy(x_0, y_0, h, k, f, n=5):
    c1, c2 = round(x_0+h, n), round(y_0+k, n)
    if type(f)==dict:
        return f[c1][c2] - f[c1][y_0] - f[x_0][c2] + f[x_0][y_0]
    elif type(f)==str:
        f = sympy.simplify(f)
        return sub_vals(f, c1, c2) - sub_vals(f, c1, y_0) - sub_vals(f, x_0, c2) + sub_vals(f, x_0, y_0)

In [13]:
def jacobian_matrix(x_vals, y_vals, f1, f2, x_0, y_0, print_table = False):
    if print_table:
        print_details(x_vals, y_vals, f1)
        print_details(x_vals, y_vals, f2)
    
    h, k = getHeight(x_vals), getHeight(y_vals)
    arr = make_table_xy(x_vals, y_vals, f1)
    dxf1 = del_x_j(x_0, y_0, h, arr)
    dyf1 = del_y_j(x_0, y_0, k, arr)

    arr = make_table_xy(x_vals, y_vals, f2)
    dxf2 = del_x_j(x_0, y_0, h, arr)
    dyf2 = del_y_j(x_0, y_0, k, arr)

    J_mat = [[dxf1, dyf1], [dxf2, dyf2]]
    
    if print_table:
        print(" Jacobian Matrix = ", end='')
        DrawMat(J_mat)
    
    return J_mat

In [14]:
def Newton_Bivariate_interpolation(x_vals, y_vals, arr, at_x='x', at_y='y', show_working=False):
    if len(x_vals)>3 or len(y_vals)>3:
        print("\n  [-] This Funtion Can only handle x and y at 3 values ...  ")
        return None
    
    x_0, y_0 = x_vals[0], y_vals[0]
    h, k = getHeight(x_vals), getHeight(y_vals)
    f = get_x_y_at(x_vals, y_vals, arr)
    if show_working: 
        print(f'Here h : {h}, k : {k}, x0 : {x_0}, y0 : {y_0}', end = ', ')
    t = ''
    for i in range(len(x_vals)):
        if i==0:
            t += f'{f[x_0][y_0]} + '

        else:
            if i == 1:
                fwd_val_x = del_x(x_0, y_0, h, f)
                fwd_val_y = del_y(x_0, y_0, k, f)
                if show_working: 
                    print(f'∆x = {fwd_val_x}, ∆y = {fwd_val_y}', end=', ')

            elif i==2:
                fwd_val_x = del_xx(x_0, y_0, h, f)
                fwd_val_y = del_yy(x_0, y_0, k, f)
                fwd_val_xy = del_xy(x_0, y_0, h, k, f)
                
                if show_working:
                    print(f'∆xx = {fwd_val_x}, ∆yy = {fwd_val_y}, ∆xy = {fwd_val_xy}')
                    
            s1 = getFactors(x_vals, i, at_x) +  ' * '
            s2 = ' + '+ getFactors(y_vals, i, at_y) + ' * '

            s1 += f'({fwd_val_x}) '
            s2 += f'({fwd_val_y}) '
            total = s1 
            if i==2:
                total += f" + 2/({h}*{k}) * {getFactors(x_vals, 1)} * {getFactors(y_vals, 1, 'y')} * ({fwd_val_xy})"
            total +=  s2
            mul = ''
            if i>1:
                mul = f'1/{fac(i)} * '

            add = ''
            if i+1 != len(x_vals): add = '+'
            t += f'{mul}({total}) {add} '
            
    if show_working:
        print('\n',t)
        
    return sympy.simplify(t)

## Index 

### 1. [Numerical Differentiation](#Numerical-Differentiation)
### 2. [Piecewise  Interpolation](#Piecewise-Interpolation)
### 3. [Lagrange's Bivariate Interpolation](#Lagranges-Bivariate-Interpolation)
### 4. [Newton's Bivariate Interpolation](#Newtons-Bivariate-Interpolation)
### 5. [Interpolation for Numerical Partial Differentiation](#Interpolation-for-Numerical-Partial-Differentiation)
### 6. [Jacobian Matrix](#Jacobian-Matrix)
### 7. [Forward Difference Value For Bivariate](#Forward-Difference-Value-For-Bivariate)

## Numerical Differentiation

In [79]:
x_values = [4.5, 5, 5.5, 6, 6.5, 7, 7.5]
f_values = [9.69, 12.9, 16.71, 21.18, 26.37, 32.34, 39.15] #[0, 0.128, 0.544, 1.296, 2.432, 4]

x_values = [0, 1.5, 2.5]
f_values = [0.621, 0.818, 0.889]

x_values, f_values = [3, 3.2, 3.4, 3.6, 3.8, 4], [-14, -10.032, -5.296, 0.256, 6.672, 14]

x_values, f_values = [1.2, 1.4, 1.6], [0.5506, 0.6048, 0.6658]
x_values, f_values = [0.4, 0.6, 0.8], [0.0256, 0.1296, 0.4096]
x_values, f_values = [1, 1.5], [0.723, 0.812]
x_values, f_values = [0, 1.5, 2.5], [0.621, 0.818, 0.889]
x_values, f_values = [0.8, 1, 1.2], [0.4096, 1, 2.0736]

x_values, f_values = [1.2, 1.4, 1.6], [0.5506, 0.6048, 0.6658]
x_values, f_values = [0.4, 0.6, 0.8], [0.0256, 0.1296, 0.4096]
x_values, f_values = [0.4, 0.5, 0.6, 0.7, 0.8], [1.5836, 1.7974, 2.0442, 2.3275, 2.6511]
DrawForwardDiffTable(x_values, f_values, rn=5)
print()
# x_values = [3, 3.2, 3.4, 3.6, 3.8, 4]
# f_values = [-14, -10.032, -5.296, 0.256, 6.672, 14]

diff_eq, h, a, x, at_x = Numerical_Diff(x_values, f_values)

n = len(" + ".join(diff_eq))
print('-'*(n+15))
print(f" ∴ f(a+xh) = {' + '.join(diff_eq)}")
print('-'*(n+15))

cal_derivative(diff_eq, h, a, x, at_x,  1) # 1st order derivative
cal_derivative(diff_eq, h, a, x, at_x, 2)  # 2nd order derivative


--------------------------------------------------------------------------------------------------
|  x   |   f(x)  |       ∆ f(x)      |      ∆^2 f(x)     |      ∆^3 f(x)     |      ∆^4 f(x)     |
--------------------------------------------------------------------------------------------------
| 0.4  |  1.5836 | ∆ f(0.4) = 0.2138 | ∆^2 f(0.4) = 0.033 | ∆^3 f(0.4) = 0.0035 | ∆^4 f(0.4) = 0.0003
--------------------------------------------------------------------------------------------------
| 0.5  |  1.7974 | ∆ f(0.5) = 0.2468 | ∆^2 f(0.5) = 0.0365 | ∆^3 f(0.5) = 0.0038 |
--------------------------------------------------------------------------------------------------
| 0.6  |  2.0442 | ∆ f(0.6) = 0.2833 | ∆^2 f(0.6) = 0.0403 |
--------------------------------------------------------------------------------------------------
| 0.7  |  2.3275 | ∆ f(0.7) = 0.3236 |
--------------------------------------------------------------------------------------------------
| 0.8  |  2.6511 |
---

In [ ]:
DrawBackDiffTable(x_values, f_values)

## Piecewise Interpolation

In [36]:
def f(x):
    return x**2 + 2* x

In [37]:
[round(f(x), 5) for x in [2, 2.2, 2.6]]

[8, 9.24, 11.96]

In [40]:
x_list, f_values, gap = [2, 2.2, 2.6], [8, 9.24, 11.96], 2

Draw_table(x_list, f_values)
Piecewise_Interpolation(x_list,f_values, gap, True)


---------------------------------------------
|    x     |    2     |   2.2    |   2.6    |
---------------------------------------------
|    y     |    8     |   9.24   |  11.96   |
---------------------------------------------

By Piecewise Quadratic Interpolating Formula we have, 
	By Lagrange's Interpolation Formula

   f(x) = 
	(x - 2.2) * (x - 2.6) /  ((2 - 2.2) * (2 - 2.6))  * (8) + 	(x - 2) * (x - 2.6) /  ((2.2 - 2) * (2.2 - 2.6))  * (9.24) + 	(x - 2) * (x - 2.2) /  ((2.6 - 2) * (2.6 - 2.2))  * (11.96)  
        = 1.00000000000003*x**2 + 1.99999999999989*x + 1.13686837721616e-13  x  ∈  [ 2, 2.6 ]
------------------------------------------------------------


In [43]:
sympy.expand('(x - 2) * (x - 2.2) /  ((2.6 - 2) * (2.6 - 2.2))  * (11.96)  ')

49.8333333333333*x**2 - 209.3*x + 219.266666666667

In [46]:
66.66667 - 115.5 + 49.83333

0.9999999999999929

## Lagranges Bivariate Interpolation

In [15]:
x_vals, y_vals = [0.1, 0.2, 0.3], [0.1, 0.2, 0.3]

In [16]:
# arr = get_f_x_y(x_vals, y_vals)
arr = [[2.02, 2.0351, 2.0403], [2.0351, 2.0801, 2.1153], [2.0403, 2.1153, 2.1893]]

In [17]:
create_xy_table(x_vals, y_vals, arr)


 ----------------------------------------------
|  y \ x   |    0.1    |   0.2    |   0.3      |
 ----------------------------------------------
|   0.1    |    2.02   |   2.0351  |   2.0403  |
------------------------------------------------
|   0.2    |   2.0351  |   2.0801  |   2.1153  |
------------------------------------------------
|   0.3    |   2.0403  |   2.1153  |   2.1893  |
------------------------------------------------




In [18]:
t = Lagrange_Bivariate_IF(x_vals, y_vals, arr, True)
exp = simplify(t)
at_x, at_y = 0.2, 0.1
# ans = expand(t).subs({'x':at_x, 'y':at_y})
# print(f' ∴ P({at_x},{at_y}) = {exp} =  {ans}')

Here m: 3, n: 3


By Lagrange's Bivariate Interpolating Formula 

P(x, y) = X_2,0.1 * [  Y_2,0.1 * (F_0.1_0.1) +   Y_2,0.2 * (F_0.1_0.2) +   Y_2,0.3 * (F_0.1_0.3)  ]
	 + X_2,0.2 * [  Y_2,0.1 * (F_0.2_0.1) +   Y_2,0.2 * (F_0.2_0.2) +   Y_2,0.3 * (F_0.2_0.3)  ]
	 + X_2,0.3 * [  Y_2,0.1 * (F_0.3_0.1) +   Y_2,0.2 * (F_0.3_0.2) +   Y_2,0.3 * (F_0.3_0.3)  ]

= ((x - 0.2) * (x - 0.3) / (0.02)) * [ ((y - 0.2) * (y - 0.3) / (0.02) * 2.02)  + ((y - 0.1) * (y - 0.3) / (-0.009999999999999998) * 2.0351)  + ((y - 0.1) * (y - 0.2) / (0.019999999999999993) * 2.0403) ]
 + ((x - 0.1) * (x - 0.3) / (-0.009999999999999998)) * [ ((y - 0.2) * (y - 0.3) / (0.02) * 2.0351)  + ((y - 0.1) * (y - 0.3) / (-0.009999999999999998) * 2.0801)  + ((y - 0.1) * (y - 0.2) / (0.019999999999999993) * 2.1153) ]
 + ((x - 0.1) * (x - 0.2) / (0.019999999999999993)) * [ ((y - 0.2) * (y - 0.3) / (0.02) * 2.0403)  + ((y - 0.1) * (y - 0.3) / (-0.009999999999999998) * 2.1153)  + ((y - 0.1) * (y - 0.2) / (0.019999999999999993) * 2.18

In [31]:
d1 = sympy.diff(exp, 'y')
sympy.diff(d1, 'x').subs({'x':0.2, 'y':0.1})

3.2175000000000124

In [34]:
sympy.diff(exp, 'x').subs({'x':0.2, 'y':0.1})

0.10150000000000239

In [35]:
sympy.diff(exp, 'y').subs({'x':0.2, 'y':0.2})

0.40100000000000301

In [40]:
x, y = 0.3, 0.3
round(3 * sin (x * y) + cos(x) + cos(y), 4)

2.1803

In [33]:

arr

[[2.02, 2.0351, 2.0403], [2.0351, 2.0801, 2.1153], [2.0403, 2.1153, 2.1803]]

## Newtons Bivariate Interpolation

In [46]:
x_vals = [0.1, 0.2, 0.3]
y_vals = [0.1, 0.2, 0.3]
arr =  [[2.02, 2.0351, 2.0403], [2.0351, 2.0801, 2.1153], [2.0403, 2.1153, 2.1803]] #[[4, 11, 30], [18, 25, 44], [56, 63, 82]]

x_vals, y_vals = [0.2, 0.3], [0.1, 0.2]
arr = [[2.0351, 2.0801], [2.0403, 2.1153]]
create_xy_table(x_vals, y_vals, arr)
ans = Newton_Bivariate_interpolation(x_vals, y_vals, arr)


 -----------------------------------
|  y \ x   |    0.2    |   0.3      |
 -----------------------------------
|   0.1    |   2.0351  |   2.0403  |
-------------------------------------
|   0.2    |   2.0801  |   2.1153  |
-------------------------------------


0.2 0.1 0.1 {0.2: {0.1: 2.0351, 0.2: 2.0801}, 0.3: {0.1: 2.0403, 0.2: 2.1153}}


KeyError: 0.30000000000000004

In [40]:
print(ans)
ans.diff('x').subs({'x':0.2, 'y':0.1})

-0.0049500000000000099*x**2 + 2.9900000000000034*x*y - 0.28241500000000049*x - 0.0049500000000000099*y**2 - 0.28241500000000049*y + 2.0466820000000001


0.014604999999999865

## Interpolation for Numerical Partial Differentiation

In [26]:
x_vals = [0.2, 0.3]
y_vals = [0.1, 0.2]
h, k = getHeight(x_vals), getHeight(y_vals)
x_at, y_at = float(input("At x: ")), float(input("At y: "))

go = True
if h is None or k is None:
    print("[!] Check the intervals. !!!")
    go = False
    
if go:
    arr = get_f_x_y(x_vals, y_vals)
    f = get_x_y_at(x_vals, y_vals, arr)

At x: 3
At y: 2
value at (0.2, 0.1) : 1
value at (0.2, 0.2) : 8
value at (0.3, 0.1) : 2
value at (0.3, 0.2) : 12


In [27]:
d_fx_f(x_vals, x_at, y_at, h, f), d_fy_f(y_vals, x_at, y_at, k, f)

ValueError: 3.0 is not in list

## Jacobian Matrix

In [47]:
x_vals = [0, 1, 2]
y_vals = [0, 1, 2]


f1 = 'x**2 + x*y - y**2'
f2 = '2*x**2 + 5*x*y + 3*y**2'



# f1 = 
f2 = '3*x**2 + 5*y**2 + x'

f1 = 'x**3 + x*y**2 - y**3'
f2 = 'x**2 - y**2 - y'


x_0, y_0 = 2, 2
# f1 = 'x**3 + x*y**2 - y**3'
# f2 = 'x*y + 5*x + 6*y'
jacobian_matrix(x_vals, y_vals,f1, f2, x_0, y_0, True)


f(x, y) = x**3 + x*y**2 - y**3

 ----------------------------------------------
|  y \ x   |     0     |    1     |    2       |
 ----------------------------------------------
|    0     |     0     |     1     |     8     |
------------------------------------------------
|    1     |     -1    |     1     |     9     |
------------------------------------------------
|    2     |     -8    |     -3    |     8     |
------------------------------------------------




f(x, y) = x**2 - y**2 - y

 ----------------------------------------------
|  y \ x   |     0     |    1     |    2       |
 ----------------------------------------------
|    0     |     0     |     1     |     4     |
------------------------------------------------
|    1     |     -2    |     -1    |     2     |
------------------------------------------------
|    2     |     -6    |     -5    |     -2    |
------------------------------------------------






IndexError: list index out of range

In [78]:
f2 = 'x*y+5*x+6*y'
f1, f2

('x**3 + x*y**2 - y**3', 'x*y+5*x+6*y')

In [77]:
x, y = 0,2
sympy.simplify(f1).subs({'x':x, 'y':y})

-8

In [ ]:
26 - 

In [46]:
arr = make_table_xy(x_vals, y_vals, f1)

In [47]:
del_x_j(1, 2, 1, arr)

5.5

## Forward Difference Value For Bivariate

In [22]:
x_vals = [0,1,2]
y_vals = [0, 1, 2]

x_0, y_0 =  2, 3 #x_vals[0], y_vals[0]
h, k = 2, 1 #getHeight(x_vals), getHeight(y_vals)

In [23]:
arr = get_f_x_y(x_vals, y_vals)

value at (0, 0) : 1
value at (0, 1) : 2
value at (0, 2) : 3
value at (1, 0) : 4
value at (1, 1) : 5
value at (1, 2) : 5
value at (2, 0) : 6
value at (2, 1) : 1
value at (2, 2) : 2


In [24]:

f = 'x**2 + y**2'

###   $∆_x  f(x_0, y_0)$

In [25]:
f'  ∆_x  f(x_0, y_0) = ∆_x f({x_0}, {y_0}) = {del_x(x_0, y_0, h, f)}'

'  ∆_x  f(x_0, y_0) = ∆_x f(2, 3) = 12'

###   $∆_y  f(x_0, y_0)$

In [26]:
f'  ∆_y  f(x_0, y_0) = ∆_y f({x_0}, {y_0}) = { del_y(x_0, y_0, k, f)}'

'  ∆_y  f(x_0, y_0) = ∆_y f(2, 3) = 7'

###   $ ∆_(xx) f(x_0, y_0) $

In [27]:
f'  ∆_xx f(x_0, y_0) = ∆_xx f({x_0}, {y_0}) = {del_xx(x_0, y_0, h, f)}' 

'  ∆_xx f(x_0, y_0) = ∆_xx f(2, 3) = 8'

###   $ ∆_(yy) f(x_0, y_0) $

In [28]:
f'  ∆f_yy (x_0, y_0) = ∆_yy f({x_0}, {y_0}) = {del_yy(x_0, y_0, k, f)}' 

'  ∆f_yy (x_0, y_0) = ∆_yy f(2, 3) = 2'

###   $ ∆_(xy)  f(x_0, y_0) $

In [47]:
f'  ∆f_xy (x_0, y_0) = ∆_xy f({x_0}, {y_0}) = {del_xy(x_0, y_0, h, k, f)}'

'  ∆f_xy (x_0, y_0) = ∆_xy f(2, 3) = 0'

In [ ]:
def LinearInterpolationFormula(x_list, f_values):
    intervals = getIntervals(x_list, 1)
    f_intervals = getIntervals(f_values, 1)
    x = symbols('x')
    t = []

    print('By Piecewise Linear Interpolating Formula we have, ')
    for _ in range(len(intervals)):
        add_poly = poly1d([0])
        print("\tBy Lagrange's Interpolation Formula\n\n   f(x) = ")
        for i in range(len(intervals[_])):
            for j in range(len(intervals[_])):
                if i!=j:
                    calc = (x-intervals[_][j]) * f_intervals[_][i] / (intervals[_][i] - intervals[_][j])
                    print(f'\t(x - {intervals[_][j]}) / ({intervals[_][i]} - {intervals[_][j]}) * {f_intervals[_][i]} = {calc}')
                    add_poly += poly1d([calc]) 
        t.append(add_poly)
        
        print('\t = ' , str(t[_].coeffs)[1:-1], ' for the interval ' , intervals[_])
        print('----'*15)
        
def QuadraticInterpolationFormula(x_list, f_values):
    intervals = getIntervals(x_list, 2)
    f_intervals = getIntervals(f_values, 2)
    n = 5
    print('By Piecewise Quadratic Interpolating Formula we have, ')
    res = []
    for _ in range(len(intervals)):
        print('\nf(x) = ')
        total = poly1d([0])
        for i in range(len(intervals[_])):
            temp_str = ''
            mul_poly = poly1d([1])
            for j in range(len(intervals[_])):
                if i!=j:
                    temp_str += f' (x - {intervals[_][j]}) / ({intervals[_][i]} - {intervals[_][j]})'
                    denon =  (intervals[_][i] - intervals[_][j])
                    mul_poly = mul_poly * poly1d([1, -intervals[_][j]]) / denon
            print(f'{temp_str} * {f_intervals[_][i]}')
            total = total + mul_poly * f_intervals[_][i]
        
        tt = intervals[_]
        print(' = ', simplify(" ".join(coef_2_str(around(total.coef[::-1], n)))), f'for the interval [{tt[0]}, {tt[-1]}]\n' )
        res.append(simplify(" ".join(coef_2_str(total.coef[::-1]))))
        print('----'*15)
        
    return res

In [86]:
x_vals, y_vals = [0, 1, 2], [0, 1, 2]
f = make_table_xy(x_vals, y_vals, f)